# EP1
## Cálculo do Conjunto de Mandelbrotem Paralelo com Pthreads e OpenMP

Nesse EP temos apenas quatro membros pois um trancou a materia:

| Nome | NUSP |
|------|------|
| Daniel Hotta | 9922700 |
| Matheus Laurentys | 9793714 |
| Pedro Gigeck | 10737136 |
| Rafael Gonçalves | 9009600 |


## Configuração do Ambiente

O ambiente será herdado do miniEP3

In [1]:
] up

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Updating `~/Área de Trabalho/EP1-HOJE/src/Project.toml`
 [no changes]
   Updating `~/Área de Trabalho/EP1-HOJE/src/Manifest.toml`
  [aaaa29a8] ↑ Clustering v0.13.4 ⇒ v0.14.0
  [b4f34e82] ↓ Distances v0.9.0 ⇒ v0.8.2
  [6a86dc24] ↑ FiniteDiff v2.3.1 ⇒ v2.3.2
  [b8a86587] ↓ NearestNeighbors v0.4.5 ⇒ v0.4.4


In [2]:
] st

Status `~/Área de Trabalho/EP1-HOJE/src/Project.toml`
  [336ed68f] CSV v0.6.2
  [a93c6f00] DataFrames v0.21.1
  [31c24e10] Distributions v0.23.2
  [7073ff75] IJulia v1.21.2
  [8314cec4] PGFPlotsX v1.2.6
  [91a5bcdd] Plots v1.3.2
  [1a8c2f83] Query v0.12.2
  [f3b207a7] StatsPlots v0.14.6



## Medição do tempo

Para medir o tempo de execução geral, usamos o script `run_measurements.sh`, com o comando `perf`, que foi passado no enunciado. Todas as medições foram feitas com 13 repetições.

Para medir o tempo de execução descontando as operações de I/O e alocação de memória, nós calculamos os tempos com algumas repetições da versão sequencial e usamos a opção do perf que separa o tempo de execução por cada comando específico do código, assim conseguimos obter o tempo gasto por cada função.

## Sobre as Implementações

A implementação sequencial `mandelbrot_seq.c` foi mantida intacta.

A implementação com OpenMP `mandelbrot_omp.c` foi bastante simples, o cálculo é igual ao sequêncial, porém o for externo foi trocado por um for paralelo do omp, compartilhando a váriavel `i_y`. Dessa forma, cada thread cuida de um intervalo de valores do `i_y`.

A implementação com Pthreads `mandelbrot_pth.c` seguiu a mesma lógica da com OMP, paralelizando o for externo. Cada thread recebe um `i_y` inicial e um `i_y` final e calcula as linhas esquivalentes a esse intervalo. A diferença é que as estruturas das threads e a divisão dos intervalos foram feitos explicitamente.

## Executando

O primeiro passo é compilar todo o codigo deste EP

In [3]:
; make

make: Nothing to be done for 'all'.


In [ ]:
Podemos rodar qualquer uma das implementações para observar as imagens, por exemplo

In [4]:
; ./mandelbrot_seq -2.5 1.5 -2.0 2.0 11500

usage: ./mandelbrot_seq c_x_min c_x_max c_y_min c_y_max image_size
examples with image_size = 11500:
    Full Picture:         ./mandelbrot_seq -2.5 1.5 -2.0 2.0 11500
    Seahorse Valley:      ./mandelbrot_seq -0.8 -0.7 0.05 0.15 11500
    Elephant Valley:      ./mandelbrot_seq 0.175 0.375 -0.1 0.1 11500
    Triple Spiral Valley: ./mandelbrot_seq -0.188 -0.012 0.554 0.754 11500


Por fim, rodamos os scripts com os experimentos

In [ ]:
; ./run_measurements.sh

In [ ]:
using Pkg
Pkg.add("DataFrames")

using DataFrames

# Build global DataFrame with the I/O and MemAlloc Times
disc = DataFrame()
logs = ["full" "elephant" "seahorse" "triple_spiral"]
for l in logs
    lT = string(l, "T")
    out = Pipe()
    cmd = pipeline(`cat times/$lT.log`, out)
    run(cmd)
    close(out.in)
    s = read(out, String)
    s = split(s, '\n')
    pop!(s)
    v = []
    aux = 0
    for j in s
        if j == "==="
            push!(v, aux)
            aux = 0
        else
            aux += parse(Float64, j)
        end
    end
    disc[!, "$l"] = v
end

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
  Installed FiniteDiff ─ v2.3.2
   Updating `~/Documentos/mac-0219/EP1/src/Project.toml`
 [no changes]
   Updating `~/Documentos/mac-0219/EP1/src/Manifest.toml`
 [no changes]
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1260


In [2]:
# Plot sequencial graphs

import Pkg
Pkg.add("Plots")
using DataFrames, Plots

# DataFrame to concat all data to write in csv
df_csv = DataFrame()
df_csv.sz = [2 ^ x for x in 4:13]

# Get the time values from the log files and saves in a string
function get_log_string(log)
    out = Pipe()
    cmd = pipeline(`grep '.seconds time elapsed.' results/mandelbrot_seq/$log.log`,
          pipeline(`awk '{print $1}'`; stdout=out))
    
    run(cmd)
    close(out.in)
    s = read(out, String)
    return s
end

# Get the percent values of the error from the log files and saves in a string
function get_log_deviation(log)
    out = Pipe()
    cmd = pipeline(`grep '.seconds time elapsed.' results/mandelbrot_seq/$log.log`,
          pipeline(`awk '{print $(NF - 1)}'`; stdout=out))
    
    run(cmd)
    close(out.in)
    s = read(out, String)
    return s
end

# Transform the strings of times and error into float arrays
function get_values(values, percent, discount)
    values = split(values, '\n')
    percent = split(percent, '\n')
    pop!(values)
    pop!(percent)
    times = []
    error = []
    for i in 1:length(values)
        t = parse(Float64, replace(values[i], ","=>"."))
        if discount != nothing
            t -= discount[i]
        end
        push!(times, t)
        e = parse(Float64, replace(replace(percent[i], ","=>"."), "%"=>""))
        push!(error, (t*e)/100)
    end
    return times, error
end

# Create a dataframe with values from the 4 log files, discount tells if we discount I/O operations
function get_data_frame(discount)
    df = DataFrame()
    df.sz = [2 ^ x for x in 4:13]
    for l in logs
        s = get_log_string(l)
        d = get_log_deviation(l)
        if discount
            dc = disc["$l"]
        else
            dc = nothing
        end
        times, dv = get_values(s, d, dc)
        df[!, "$l"] = times
        df[!, "dv$l"] = dv
        df_csv[!, "seq_$l.times"] = times
        df_csv[!, "seq_$l.error"] = dv
    end
    return df
end

function plot_data_frame(df, d)
    xs = df.sz
    ys = [df.full df.elephant df.seahorse df.triple_spiral]
    deviation = [df.dvfull df.dvelephant df.dvseahorse df.dvtriple_spiral]
    plot(xs, ys, label = logs, yerror = deviation, 
         xlabel = "Tamanho da Imagem", ylabel = "Tempo (s)", title = "Tempo x Tamanho\nImplementação Sequencial\n$d")
end

df = get_data_frame(false)
plot_data_frame(df, "Sem desconto de I/O")


   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %

  Resolving package versions...


Pkg.Types.PkgError: Error when installing package FiniteDiff:
InterruptException:
Stacktrace:
 [1] download(::String, ::String; verbose::Bool, auth_header::Nothing) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Pkg/src/PlatformEngines.jl:843
 [2] install_archive(::Array{Pair{String,Bool},1}, ::Base.SHA1, ::String) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Pkg/src/Operations.jl:492
 [3] macro expansion at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Pkg/src/Operations.jl:680 [inlined]
 [4] (::Pkg.Operations.var"#43#46"{Bool,Pkg.Types.Context,Dict{Base.UUID,Array{String,1}},Channel{Any},Channel{Any}})() at ./task.jl:358

In [ ]:
df = get_data_frame(true)
plot_data_frame(df, "Com desconto de I/O")

In [ ]:
# Plot the paralell graphs

import Pkg
Pkg.add("Plots")
using DataFrames, Plots

# Get the time values from the log files and saves in a string
function get_log_string(log, implementation, th)
    out = Pipe()
    cmd = pipeline(`grep '.seconds time elapsed.' results/mandelbrot_$implementation/$log$th.log`,
          pipeline(`awk '{print $1}'`; stdout=out))
    
    run(cmd)
    close(out.in)
    s = read(out, String)
    return s
end

# Get the percent values of the error from the log files and saves in a string
function get_log_deviation(log, implementation, th)
    out = Pipe()
    cmd = pipeline(`grep '.seconds time elapsed.' results/mandelbrot_$implementation/$log$th.log`,
          pipeline(`awk '{print $(NF - 1)}'`; stdout=out))
    
    run(cmd)
    close(out.in)
    s = read(out, String)
    return s
end

# Get a dataFrame (time x image_size) with data  
function get_data_frame(implementation, image)
    df = DataFrame()
    df.sz = [2 ^ x for x in 4:13]
    th = [2 ^ x for x in 0:4]
    for t in th
        s = get_log_string(image, implementation, string(t))
        d = get_log_deviation(image, implementation, string(t))
        
        dc = disc["$image"]
        
        times, dv = get_values(s, d, dc)
        df[!, "t$t"] = times
        df[!, "dv$t"] = dv
        df_csv[!, "$implementation.$image$t.times"] = times
        df_csv[!, "$implementation.$image$t.error"] = dv
    end
    return df
end

# Função para plotar o DataFrame
function plot_data_frame(df, i, j)
    xs = df.sz
    ys = [df.t1 df.t2 df.t4 df.t8 df.t16 df.t32]
    deviation = [df.dv1 df.dv2 df.dv4 df.dv8 df.dv16 df.dv32]
    labels = ["1" "2" "4" "8" "16" "32"]
    display(plot(xs, ys, label = labels, yerror = deviation, 
         xlabel = "Tamanho da Imagem", ylabel = "Tempo (s)", title = "Tempo x Tamanho\nImplementação $i - $j"))
end

implementations = ["omp" "pth"]
file = ["triple_spiral","seahorse","elephant","full"]
for i in implementations
    for j in file
        df = get_data_frame(i, j)
        plot_data_frame(df, i, j)
    end
end

In [ ]:
using CSV

CSV.write("data", df_csv)

### Análise da implementação sequencial

Com a implementação sequencial, vemos uma clara diferença entre as regiões do Conjunto de Mandelbrot. 

A geração da imagem completa (full) consome muito menos tempo que as regiões específicas, uma das causas pode ser o grande intervalo de pontos que cada pixel da imagem abrange, causando uma baixa definição e, com isso, menos iterações de cálculo. As regiões específicas, que cobriam subconjuntos de tamanhos semelhantes consumiram tempos parecidos. 

Outra observação interessante é o comportamento quadrático do consumo de tempo em função do tamanho da lateral imagem, o que faz sentido já que calculamos um dado número de iterações para cada pixel.

### Análise geral das implementações paralelas

No entanto,com as implementeções paralelas conseguimos observar que ,aparentemente, o Conjunto de Mandelbrot paralelizado por
pthreads apresenta um menor tempo para o calculo quando temos 8 threads. 

Além disso, fica evidente uma diferença gritante na maneira em que a evolução do tempo para o calculo das imagens se comporta independente do numero de threads involvido, dado a implementação ser por OpenMP e Pthreads, pois os graficos gerados pela implementação OpenMP apresentam um tempo de execução e taxa de crescimento similar para n threads para as regiões pedidas, enquanto ao nos voltarmos para os graficos gerados pela implementação pthreads temos taxas de crescimento para o tempo de execução por numero de threads mais distintas entre si.


### Análise da implementação com *pthreads*

Aparentemente o tempo de execução aumenta ainda que aumentamos o numero de threads, com exceção de quando rodamos
imagens com 8 threads provavelmente pois , assim como ocorreu no miniep anterior, a quantidade de threads acaba 
ficando muito maior que a quantidade de núcleos de processamento da máquina, assim, não temos um ganho real 
dividindo o trabalho, pois ele terá que ser executado nos mesmos núcleos. Todavia ainda vemos que ainda ganhamos tempo se comparado com a execução sequencial e que assim como na execução sequencial a geração da imagem completa(full) é mais rapida de ser calculada.

### Análise da implementação com OpenMP

Como ja foi dito, a implementação por OpenMP apresentam um tempo de execução e taxa de crescimento similar para n threads para as regiões pedidas, todavia apresentam um tempo consideravelmente mais rapido para determinadas imagens com relação a outras, em especial a imagem denominada " full ".

• 2) Qual o impacto das operações de I/O e alocação de memória no tempo de execução?

#### Especificações do ambiente utilizado para geração das imagens 

 - Host bridge: Intel Corporation Xeon E3-1200 v6/7th Gen Core Processor Host Bridge/DRAM Registers 
 
 - PCI bridge: Intel Corporation Xeon E3-1200 v5/E3-1500 v5/6th Gen Core Processor PCIe Controller (x16) (rev 05)
 
 - VGA compatible controller: Intel Corporation Device 591b (rev 04)
 
 - USB controller: Intel Corporation 100 Series/C230 Series Chipset Family USB 3.0 xHCI Controller (rev 31)
  
 - Signal processing controller: Intel Corporation 100 Series/C230 Series Chipset Family Thermal Subsystem (rev 31)
 
 - Communication controller: Intel Corporation 100 Series/C230 Series Chipset Family MEI Controller #1 (rev 31)
 
 - SATA controller: Intel Corporation HM170/QM170 Chipset SATA Controller [AHCI Mode] (rev 31)
 
 - PCI bridge: Intel Corporation 100 Series/C230 Series Chipset Family PCI Express Root Port #3 (rev f1)
 
 - PCI bridge: Intel Corporation 100 Series/C230 Series Chipset Family PCI Express Root Port #4 (rev f1)
 
 - PCI bridge: Intel Corporation 100 Series/C230 Series Chipset Family PCI Express Root Port #9 (rev f1)
 
 - ISA bridge: Intel Corporation HM175 Chipset LPC/eSPI Controller (rev 31)
 
 - Memory controller: Intel Corporation 100 Series/C230 Series Chipset Family Power Management Controller (rev 31)
 
 - SMBus: Intel Corporation 100 Series/C230 Series Chipset Family SMBus (rev 31)
 
 - VGA compatible controller: NVIDIA Corporation GP106M [GeForce GTX 1060 Mobile] (rev a1)
 
 - Network controller: Intel Corporation Wireless 8265 / 8275 (rev 78)
 
 - Ethernet controller: Realtek Semiconductor Co., Ltd. RTL8111/8168/8411 PCI Express Gigabit Ethernet Controller


In [ ]:
using DataFrames

function get_df_time(path) 
    overhead = []
    system = []
    open(path) do f
        for i in eachline(f)
            if occursin(r".+[0-9]+.[0-9]+%", i)
                x = split(i)
                tmp = split(x[1], "%")
                push!(overhead, parse(Float64, tmp[1]))
                push!(system, x[5])
            end
        end
    end
    return df = DataFrame(A = overhead, B = system)
end


df = (get_df_time("results_inter/mandelbrot_seq/full.log"))


function get_time(df)
    time = [0.0, 0.0, 0.0]

    for i in eachrow(df)
        if occursin(r".*mandelbrot.*", i[2,])
            time[1] += i[1, ]
        elseif occursin(r".*malloc.*", i[2,]) || occursin(r".*IO.*", i[2,]) 
            time[2] += i[1, ]
        else 
            time[3] += i[1, ]
        end
    end
    return time
end

println(typeof(get_time(df)))


function get_all()
    mandel = ["omp", "pth"]
    logs = ["elephant8", "full8", "seahorse8", "triple_spiral8"]
    
    #mandelbrot
    timeM = []
    
    #IO and malloc
    timeIO = []
    
    #Remain
    timeR = []
    
    files = []
    
    for i in mandel
        for j in logs
            tmp_df = (get_df_time("results_inter/mandelbrot_$i/$j.log"))
            time = get_time(tmp_df)
            push!(timeM, time[1])
            push!(timeIO, time[2])
            push!(timeR, time[3])
            push!(files, "$i/$j")
        end
    end
    
    return df = DataFrame(Mandelbrot = timeM, IOandMalloc = timeIO, Remain = timeR, Files = files)
end

println(get_all())